# Neural Networks

In [39]:
import os
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

'2'

In [40]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
from scipy import stats

## Data

In [41]:

# Load the CSV file
data = pd.read_csv("BC-Data-Set.csv")

In [42]:
# Convert the date column to a datetime object
data['date'] = pd.to_datetime(data['date'])

# Remove any missing values
data = data.dropna()

# Set the date column as the index of the DataFrame
data = data.set_index('date')

In [43]:
seed = 42

In [44]:
from sklearn.preprocessing import StandardScaler

In [45]:
import tensorflow as tf
from tensorflow import keras

## Neural Network (non-LSTM)

In [46]:
x_train, x_test, y_train, y_test = train_test_split(data, data.BC, test_size=0.15, random_state=seed, shuffle=True)
x_train = x_train.drop(columns=['BC'])
x_test = x_test.drop(columns=['BC'])

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=seed, shuffle=True)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [47]:
HIDDEN_DIM = 8
inputs = tf.keras.layers.Input(shape=[x_train.shape[-1]])

layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(inputs)
layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(layer)

predictions = keras.layers.Dense(1, activation=None)(layer)

model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=tf.losses.mean_squared_error,
            metrics=[tf.metrics.mean_squared_error],
        )


In [48]:
batch_size = 32
epochs = 20
logs = model.fit(
    x_train, y_train,
    batch_size=batch_size, epochs=epochs,
    validation_data=(x_val, y_val),
    # callbacks=[model.tb_callback],
)

Epoch 1/20
96/96 [==============================] - 1s 2ms/step - loss: 2.5651 - mean_squared_error: 2.5651 - val_loss: 1.8840 - val_mean_squared_error: 1.8840
Epoch 2/20
96/96 [==============================] - 0s 1ms/step - loss: 1.0976 - mean_squared_error: 1.0976 - val_loss: 0.9429 - val_mean_squared_error: 0.9429
Epoch 3/20
96/96 [==============================] - 0s 1ms/step - loss: 0.6983 - mean_squared_error: 0.6983 - val_loss: 0.7631 - val_mean_squared_error: 0.7631
Epoch 4/20
96/96 [==============================] - 0s 1ms/step - loss: 0.5973 - mean_squared_error: 0.5973 - val_loss: 0.7008 - val_mean_squared_error: 0.7008
Epoch 5/20
96/96 [==============================] - 0s 1ms/step - loss: 0.5559 - mean_squared_error: 0.5559 - val_loss: 0.6527 - val_mean_squared_error: 0.6527
Epoch 6/20
96/96 [==============================] - 0s 1ms/step - loss: 0.5285 - mean_squared_error: 0.5285 - val_loss: 0.6259 - val_mean_squared_error: 0.6259
Epoch 7/20
96/96 [======================

## LSTM

In [49]:
data

,BC,N_CPC,PM-10,PM-2.5,PM-1.0,NO2,O3,SO2,CO,NO,NOX,TEMP,HUM
date,,,,,,,,,,,,,
2019-01-01 00:00:00,3.445,17.773,13.033,11.767,11.050,18.0,41.0,1.0,0.2,4.0,23.0,12.140,53.780
2019-01-01 01:00:00,5.162,24.620,17.050,15.533,14.783,62.0,1.0,1.0,0.2,22.0,96.0,11.683,55.100
2019-01-01 02:00:00,2.951,17.700,11.417,10.367,9.850,68.0,1.0,1.0,0.2,42.0,132.0,11.683,53.433
2019-01-01 03:00:00,1.917,12.212,7.750,7.100,6.683,75.0,1.0,1.0,0.2,66.0,176.0,11.367,54.533
2019-01-01 04:00:00,2.462,11.178,8.317,7.650,7.167,63.0,1.0,1.0,0.2,27.0,103.0,10.867,57.850
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-10 19:00:00,0.728,9.275,7.300,5.267,3.267,30.0,57.0,1.0,0.2,2.0,33.0,22.033,93.367
2019-12-10 20:00:00,0.787,9.420,7.500,5.467,3.417,37.0,46.0,1.0,0.2,1.0,38.0,22.100,93.317
2019-12-10 21:00:00,1.056,9.296,7.233,5.283,3.200,16.0,67.0,1.0,0.2,1.0,17.0,22.200,93.333


In [60]:
SEQUENCE_LENGTH = 12
SEQUENCE_DIM = x_train.shape[-1]
X = np.random.normal(size=[1000, SEQUENCE_LENGTH, SEQUENCE_DIM])
Y = np.random.normal(size=[1000, SEQUENCE_LENGTH, 1])
X.shape, Y.shape

((1000, 12, 12), (1000, 12, 1))

In [14]:
# TODO
def build_sequences(df, window=200, stride=200):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    for id in df['id'].unique():
        # Take only meaningful features
        temp = df[df['id'] == id][['x_axis','y_axis','z_axis']].values
        # Save the label
        label = df[df['id'] == id]['activity'].values[0]
        print(str(id), "  ", str(label))
        # Compute padding length
        padding_len = window - len(temp)%window
        # Create padding and concatenate it
        padding = np.zeros((padding_len,3), dtype='float64')
        temp = np.concatenate((temp,padding))
        # Build features windows with their corresponging labels
        idx = 0
        while idx+window <= len(temp):
            dataset.append(temp[idx:idx+window])
            labels.append(label)
            idx += stride
    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [61]:
RNN_CELL_DIM = 8
HIDDEN_DIM = 8
sequences = tf.keras.layers.Input(shape=[SEQUENCE_LENGTH, SEQUENCE_DIM])

layer = keras.layers.LSTM(RNN_CELL_DIM, return_sequences=True)(sequences)

layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(layer)

predictions = keras.layers.Dense(1, activation=None)(layer)

model = tf.keras.Model(inputs=sequences, outputs=predictions)
model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=tf.losses.mean_squared_error,
            metrics=[tf.metrics.mean_squared_error],
        )


In [62]:
batch_size = 32
epochs = 10
logs = model.fit(
    X, Y,
    batch_size=batch_size, epochs=epochs,
    validation_data=(X, Y),
    # callbacks=[model.tb_callback],
)

Epoch 1/10
32/32 [==============================] - 2s 17ms/step - loss: 1.0326 - mean_squared_error: 1.0326 - val_loss: 1.0231 - val_mean_squared_error: 1.0231
Epoch 2/10
32/32 [==============================] - 0s 7ms/step - loss: 1.0208 - mean_squared_error: 1.0208 - val_loss: 1.0181 - val_mean_squared_error: 1.0181
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 1.0170 - mean_squared_error: 1.0170 - val_loss: 1.0153 - val_mean_squared_error: 1.0153
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 1.0147 - mean_squared_error: 1.0147 - val_loss: 1.0134 - val_mean_squared_error: 1.0134
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 1.0132 - mean_squared_error: 1.0132 - val_loss: 1.0121 - val_mean_squared_error: 1.0121
Epoch 6/10
32/32 [==============================] - 0s 7ms/step - loss: 1.0120 - mean_squared_error: 1.0120 - val_loss: 1.0111 - val_mean_squared_error: 1.0111
Epoch 7/10
32/32 [=====================